In [0]:
%run /Workspace/Users/saijananig4@gmail.com/real-time-banking-etl/src/config/00_config_and_helpers

In [0]:
dbutils.widgets.text("env", "dev")
env = dbutils.widgets.get("env")

cfg = get_env_config(env)

BRONZE_TBL = tbl(cfg, "bronze_transactions")
SILVER_TBL = tbl(cfg, "silver_transactions")

print("Running env:", cfg["ENV"])
print("BRONZE_TBL:", BRONZE_TBL)
print("SILVER_TBL:", SILVER_TBL)


In [0]:
from pyspark.sql import functions as F

df_bronze = spark.read.table(BRONZE_TBL)

print("Bronze count:", df_bronze.count())
df_bronze.show(10, truncate=False)


In [0]:
df_silver = (
    df_bronze
    .filter(F.col("amount") >0)
    .withColumn("transaction_ts", F.date_format(F.col("transaction_ts"), "yyyy-MM-dd HH:mm:ss"))
    .withColumn("transaction_date", F.to_date("transaction_ts"))
    .withColumn("hour_of_day", F.hour("transaction_ts"))
    .withColumn("day_of_week", F.date_format("transaction_ts", "EEEE"))
    .withColumn("is_high_value",
                F.when(F.col("amount") > 5000, F.lit(True)).otherwise(F.lit(False))
            )
)

df_silver.display()

In [0]:


(df_silver.write
 .format("delta")
 .mode("overwrite")
 .option("mergeSchema", "true")
 .saveAsTable(SILVER_TBL))

print("✅ Silver table updated:", SILVER_TBL)
print("Silver count:", spark.read.table(SILVER_TBL).count())


In [0]:
%sql 

SELECT
  COUNT(*) AS total_rows
from finance_data.dev.silver_transactions;
